## <center> Numerical computation and visualization of the  Laplacian eigenfunctions<br><br>Button `on_click` callback function </center>

The numerical computation of eigenvalues of differential operators is
usually performed following two main philosophies. On the one hand,
one starts out with a discretization of the differential operator by a
finite dimensional matrix approximation obtained by finite differences,
finite elements, or any other numerical method. The spectrum of the
approximation is then taken as an approximation to the spectrum of the
original differential operator.

The properties of Laplacian eigenvalues and eigenfunctions have been investigated in many scientific
disciplines,  theory of acoustical, optical, and quantum waveguides, condensed
matter physics and quantum mechanics, quantum graphs, image processing, computer
graphics,  biology and more.

Nowadays,  eigenfunctions of the Laplace operator have applications in Data Science and Machine Learning,
for face recognition, clustering,  nonlinear image denoising and segmentation,  mapping of protein energy landscapes, and more.
A google search reveals more examples.

To get an idea of their properties,  we illustrate here how to compute the spectrum of the Laplace operator, $\Delta u=\lambda u,$ 
$$\Delta=\displaystyle\frac{\partial^2}{\partial x^2}+\displaystyle\frac{\partial^2}{\partial y^2},$$ 
on a bounded planar domain.

The bounded domain is represented by a binary np.array, with 0 inside the domain and 1 outside it.

In this example we take the Maui island [https://mauiguide.com/maps/](https://mauiguide.com/maps/) as a bounded irregular 2d domain. A png image of this island was previously processed in order to extract a representative binary array.

In [1]:
import platform
import plotly
print(f'Python version: {platform.python_version()}')
print(f'Plotly version: {plotly.__version__}')

Python version: 3.7.1
Plotly version: 3.8.1


In [2]:
import numpy as np 
import scipy.sparse
from scipy.sparse import linalg
import plotly.graph_objs as go
import ipywidgets as ipw

Read the binary array previously created and saved as a `npy` file:

In [3]:
bimg = np.load('maui.npy') #https://github.com/empet/Mathematical-Physics/blob/master/data/maui.npy
if bimg.shape[0] != bimg.shape[1]:
    raise ValueError('The code below works with arrays of shape (n,n)')
vals = np.unique(bimg)  
if len(vals) != 2 or 0 or 1 not in vals:
    raise ValueError('Your array is not binary')
bimg = np.array(bimg, float)
bimg = np.flipud(bimg)

Perform the numerical discretization of the Helmholz equation $\Delta u=-\lambda u$ with Dirichlet boundary condition
 on this special domain.
 
For a better performance we are working with `scipy.sparse` matrices.

In [4]:
newimg = bimg + 1e-05 # a slight perturbation of the binary array
n = newimg.shape[0]
e = np.ones(n)
diagonal_vals = np.array([-e, e, -e])
partial_x = scipy.sparse.spdiags(diagonal_vals, np.array([-1, 0, n-1]), n, n)

gradient = scipy.sparse.vstack([scipy.sparse.kron(partial_x, scipy.sparse.eye(n)), 
                                scipy.sparse.kron(scipy.sparse.eye(n), partial_x)])

img_as_row = np.concatenate((newimg.flatten(), newimg.flatten()))
Laplace = gradient.transpose()*scipy.sparse.spdiags(img_as_row, [0], 2*n**2, 2*n**2)*gradient 
k = 12

Compute the k smalllest eigenvalues and eigenfunctions of the Laplace operator, [https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.eigs.html](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.linalg.eigs.html),  with shift-invert mode corresponding to `sigma`=0.
In this case the transformed eigenvalues are `eigvals'=1/(eigvals-sigma)=1/eigvals`.
With the option `which='LM'` we get the k largest `eigvals'`, i.e.  the smallest 
eigenvalues of the original problem.

`Laplace.todense()` is a symmetric positive matrix of dimension $n^2 \times n^2$. The eigenvalues of the Laplace matrix  are ordered as follows:

$$0\leq\lambda_0\leq \lambda_1 \leq \cdots \lambda_{n^2-1}$$

In [5]:
Laplace.todense().shape

(22500, 22500)

Extract the first k=12 eigenvalues and eigenvectors (discretized eigenfunctions):

In [6]:
eigvals, u = scipy.sparse.linalg.eigsh(Laplace, k, which='LM', sigma=0) 

for j in range(k):
    print( f"Eigenvalue {j}: " +"{:.8f}".format(eigvals[j]) )

Eigenvalue 0: 0.00000000
Eigenvalue 1: 0.00000004
Eigenvalue 2: 0.00000007
Eigenvalue 3: 0.00000008
Eigenvalue 4: 0.00000011
Eigenvalue 5: 0.00000013
Eigenvalue 6: 0.00000014
Eigenvalue 7: 0.00000016
Eigenvalue 8: 0.00000017
Eigenvalue 9: 0.00000021
Eigenvalue 10: 0.00000021
Eigenvalue 11: 0.00000025


In [7]:
eigenf = np.zeros((n, n, k)) #an array consisting in  stacked eigenfunctions
for i in range(k):
    uu = u[:,i].reshape((n, n)) #reshape the i^th eigenvector to create a contour trace from it
    uu = uu / np.max(np.fabs(uu)) # normalize the image representing the eigenfunction u[:,i]
    uu[bimg==1] = np.nan
    eigenf[:, :, i] = uu 

In [8]:
pl_RdYlBu = [[0.0, 'rgb(49, 54, 149)'],
            [0.1, 'rgb(68, 115, 179)'],
            [0.2, 'rgb(116, 173, 209)'],
            [0.3, 'rgb(169, 216, 232)'],
            [0.4, 'rgb(224, 243, 248)'],
            [0.5, 'rgb(254, 254, 190)'],
            [0.6, 'rgb(254, 224, 144)'],
            [0.7, 'rgb(252, 172, 96)'],
            [0.8, 'rgb(244, 109, 67)'],
            [0.9, 'rgb(214, 47, 38)'],
            [1.0, 'rgb(165, 0, 38)']]


Define the contour  trace representing the eigenfunction corresponding to the lowest eigenvalue=0. This eigenfunction is constant on the chosen domain. Hence its `go.FigureWidget` instance is not so interesting like those corresponding to nearby eigenvalues.

In [9]:
trace = dict(type='contour',
             z=bimg.tolist(), # since this eigenfucntion is constant it can be represented by bimg
             colorscale=pl_RdYlBu,
             reversescale=True,
             colorbar=dict(thickness=20, ticklen=4), 
             contours=dict(start=-1, end=1, size=0.125), 
             line=dict(width=0.5, color='rgb(200,200,200)'))

In [10]:
layout = dict(title=f"Maui island",
              font=dict(family='Balto',
                        size=12),
              width=500, height=500, 
              xaxis=dict(visible=False), 
              yaxis=dict(visible=False), 
              plot_bgcolor='rgb(0, 101, 93)')#'rgb(49, 54, 149)')

fw = go.FigureWidget(data=[trace], layout=layout)

Define a button widget and a callback function that `on_click` updates the `FigureWidget` to get the eigenfunction of the next eigenvalue
in the list of 12 eigenvalues computed above:

In [11]:
button=ipw.Button(description='Click me!',
                  layout=dict(margin='-50px 10px 10px 80px', width='100px') )

i=0
def click(b):
    global i, eigvals
    i += 1
    j = i % 12
    with fw.batch_animate():
        fw.data[0].z = eigenf[:, :, j].tolist()
        fw.layout.title = f"Eigenfunction of the Laplace operator<br>corresponding to eigval={eigvals[j]:0.8f}"    
            
button.on_click(click)
ipw.VBox([fw, button])

    'data': [{'colorbar': {'thickness': 20, 'ticklen': 4},
              'colors…

To see the above code in action just run this notebook. Here we illustrate the image of the eigenfunction corresponding to
the eigenvalue $\lambda=0.00000014$:

In [12]:
from IPython.display import IFrame
IFrame('https://plot.ly/~empet/14997', width=700,  height=500)

Animation of  the succesive plots of eigenfunctions generated on click, in the code above:

In [13]:
%%html
<img src='laplace-maui.gif'>

In [ ]:
.